In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [1]:
%run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

data = Data('s', accent_classification=True)
# data.generate_data('shuffled_syllable_reversed_train',
#                    'shuffled_syllable_reversed_test',
#                    'shuffled_syllable_reversed_validate')
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate', force_override=False)

Using Theano backend.


LOADING DATA...


MemoryError: 

In [4]:
%run ../../../prepare_data.py
# gen1 = generate_fake_epoch_syllables(X_train[:20], X_other_features_train[:20], y_train[:20], 16, 5168)
# gen2 = generate_fake_epoch_syllables(data.x_train[:20], data.x_other_features_train[:20], data.y_train[:20], 16, 5168)
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
pos = 0
# print(np.array(test1[0][0]).shape)
# for el in test1[0][0][pos]:
#     print(el)
#     print(syllable_dictionary[el.tolist().index(1)])
# print('-----------------------------------------------')
print(np.array(test2[0][0]).shape)
for el in test2[0][0][pos]:
    print(el)
    print(syllable_dictionary[el.tolist().index(1)])
    
print(data.decode_y(data.y_train[pos]))
print(data.y_train[pos])

(16, 10, 5168)
[0 0 0 ..., 0 0 0]
ša
[0 0 0 ..., 0 0 0]
lnej
[0 0 1 ..., 0 0 0]
a
[0 0 0 ..., 0 0 0]
ktu
[0 0 0 ..., 0 0 0]
ja
[0 0 0 ..., 0 0 0]
na
[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[2, 5]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]


In [16]:
num_examples = len(data.x_train) # training set size

nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 1
num_fake_epoch = 20



conv_input_shape=(10, 5168)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(44, (2), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(44, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)





model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [17]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/20
1345/1344 [==============================] - 135s - loss: 0.1894 - actual_accuracy: 0.3905 - val_loss: 0.1265 - val_actual_accuracy: 0.6591
Epoch 2/20
 550/1344 [===========>..................] - ETA: 83s - loss: 0.1320 - actual_accuracy: 0.6082

KeyboardInterrupt: 

In [18]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [6]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)